# Artificial Neural Network

### Importing the libraries

In [108]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [109]:
tf.__version__

'2.11.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [110]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [111]:
X[0]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [112]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [113]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

### Encoding categorical data

Label Encoding the "Gender" column

In [114]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [115]:
X.shape

(10000, 10)

In [116]:
X[0]

array([619, 'France', 0, 42, 2, 0.0, 1, 1, 1, 101348.88], dtype=object)

One Hot Encoding the "Geography" column

### 1. **Nominal Categorical Features**:
   - **One-Hot Encoding** is ideal for **nominal** categorical features, where the categories do not have any intrinsic order or ranking.
   - **Example**: Consider a feature called `Color` with values `['red', 'blue', 'green']`. These colors do not have any order or ranking. Using One-Hot Encoding, each color will be represented as a separate binary feature:
     - `red` → [1, 0, 0]
     - `blue` → [0, 1, 0]
     - `green` → [0, 0, 1]

   This approach avoids introducing any unintended ordinal relationships that might mislead the model.

### 2. **Multiple Categories with Equal Importance**:
   - If your categorical feature has multiple categories that are all equally important, One-Hot Encoding ensures that each category is treated independently.
   - **Example**: A feature `VehicleType` with categories `['car', 'bus', 'bike']` should not imply that `car` < `bus` < `bike` or any other order. One-Hot Encoding represents these as separate, independent features.

### 3. **Non-Ordinal Data**:
   - When your categorical data is **non-ordinal** (i.e., the categories don't have a natural order), One-Hot Encoding is usually preferred. 
   - **Example**: For a feature `Fruit` with values `['apple', 'banana', 'cherry']`, there is no inherent order among the fruits. One-Hot Encoding avoids giving the model the impression that one fruit is "greater" or "lesser" than another.

### 4. **Avoiding Ordinal Misinterpretation**:
   - If you use Label Encoding on a non-ordinal feature, many machine learning models might interpret the encoded integers as having some order or ranking, which could lead to incorrect conclusions.
   - **Example**: If you label encode `['red', 'blue', 'green']` to `[0, 1, 2]`, the model might incorrectly assume that `red < blue < green`, which is not true. One-Hot Encoding would prevent this issue by treating each category separately.

### 5. **Compatibility with Tree-Based Models**:
   - **One-Hot Encoding** is particularly useful when working with tree-based models (like Decision Trees, Random Forests), which can interpret the presence or absence of a category feature correctly when it's represented as a binary feature.
   - **Example**: In a Decision Tree model, if you One-Hot Encode the feature `City` with values `['New York', 'Los Angeles', 'Chicago']`, the tree can split on each binary feature independently, making better decisions.

### When **Not** to Use One-Hot Encoding

1. **High Cardinality Features**:
   - If a categorical feature has a very large number of unique categories (high cardinality), One-Hot Encoding might create an excessive number of features, leading to the **curse of dimensionality**.
   - **Example**: If you have a feature `ZipCode` with hundreds of unique values, One-Hot Encoding could result in hundreds of binary features, making the model cumbersome and slow. In such cases, alternative techniques like **Target Encoding** or **Frequency Encoding** might be more appropriate.

2. **Ordinal Data**:
   - If your categorical data is **ordinal** (i.e., the categories have a meaningful order), Label Encoding might be more appropriate.
   - **Example**: For a feature `Size` with values `['small', 'medium', 'large']`, there is a natural order, so Label Encoding makes sense (e.g., `small=0`, `medium=1`, `large=2`).

In [75]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [76]:
X[0]

array([1.0, 0.0, 1.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

### Splitting the dataset into the Training set and Test set

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [103]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [79]:
X_test[0]

array([-1.75486502,  1.75486502, -1.01460667, -0.57369368, -0.55204276,
       -1.09168714, -0.36890377,  1.04473698,  0.8793029 , -0.92159124,
        0.64259497,  0.9687384 ,  1.61085707])

In [80]:
X_test

array([[-1.75486502,  1.75486502, -1.01460667, ...,  0.64259497,
         0.9687384 ,  1.61085707],
       [ 0.5698444 , -0.5698444 ,  0.98560362, ...,  0.64259497,
        -1.03227043,  0.49587037],
       [ 0.5698444 , -0.5698444 , -1.01460667, ...,  0.64259497,
         0.9687384 , -0.42478674],
       ...,
       [ 0.5698444 , -0.5698444 , -1.01460667, ...,  0.64259497,
        -1.03227043,  0.71888467],
       [-1.75486502,  1.75486502, -1.01460667, ...,  0.64259497,
         0.9687384 , -1.54507805],
       [-1.75486502,  1.75486502, -1.01460667, ...,  0.64259497,
        -1.03227043,  1.61255917]])

## Part 2 - Building the ANN

### Initializing the ANN

In [81]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [82]:
# Builds a shallow nn
# units is the number of neurons
# relu == activation function (recifier)
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

### Adding the second hidden layer

In [83]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

### Adding the output layer

In [84]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

## Part 3 - Training the ANN

### Compiling the ANN

In [85]:
# optimizer: stochastic gradient descent 
# loss: cost function (for binary)
# metrics: a list of metrics you want to measure as training
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

### Training the ANN on the Training set

In [86]:
epochs = 10
ann.fit(X_train, y_train, batch_size=32, epochs=epochs)

Epoch 1/10
250/250 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7526
Epoch 2/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4663 - accuracy: 0.7960
Epoch 3/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4471 - accuracy: 0.8000
Epoch 4/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4353 - accuracy: 0.8041
Epoch 5/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4253 - accuracy: 0.8105
Epoch 6/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4162 - accuracy: 0.8163
Epoch 7/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4092 - accuracy: 0.8217
Epoch 8/10
250/250 [==============================] - 0s 1ms/step - loss: 0.4030 - accuracy: 0.8246
Epoch 9/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3983 - accuracy: 0.8256
Epoch 10/10
250/250 [==============================] - 0s 1ms/step - loss: 0.3938 - accuracy: 0.8282

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [124]:
y_hw = ann.predict(sc.transform([[1,0,1,0, 600,1,40,3,60000,2,1,1,50000]]))

1/1 [==============================] - 0s 22ms/step


In [125]:
y_hw

array([[0.]], dtype=float32)

In [129]:
y_hw > 0.5

array([[False]])

**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [140]:
y_pred = [1 if res > .50 else 0 for res in ann.predict(X_test)]

63/63 [==============================] - 0s 2ms/step


In [141]:
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


### Making the Confusion Matrix

In [142]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1512   83]
 [ 260  145]]


0.8285

In [143]:
y_test

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)